### Inference on Donut for Visual Question Answering(DOCVQA)
Non va da sistemare

In [8]:
import re
from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch
import sentencepiece as spm
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-to-text", model="naver-clova-ix/donut-base-finetuned-cord-v2")

#processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

device="cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

dataset= load_dataset("hf-internal-testing/example-documents", split="test")
image= dataset[0]["image"]

# prepare decoder inputs
task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
question = "When is the coffee break?"
prompt = task_prompt.replace("{user_input}", question)
decoder_input_ids = pipe.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
pixel_values = pipe(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    pad_token_id=pipe.tokenizer.pad_token_id,
    eos_token_id=pipe.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[pipe.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

sequence = pipe.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(pipe.tokenizer.eos_token, "").replace(pipe.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
print(pipe.token2json(sequence))


ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.